# Recurrent Neural Network Grammars

## About this notebook

These notes were created as a presentation aid for the paper [Recurrent Neural Network Grammars](https://arxiv.org/abs/1602.07776), Dyer et al. 2016, at the [Berlin Machine Learning seminar](http://building-babylon.net/berlin-machine-learning-learning-group/). All images are from Dyer et al. 2016. For more information about code sources and licensing, please see the project README. All mistakes are, naturally, my own.

### Generative Grammar

Traditionally defined:

<!-- G = (N, Sig, P, S)-->

<!-- img src="img/grammar.png",width=200,height=200 --->

\begin{align}
G = (N, \Sigma, P, <.>)
\end{align}

* N: set of nonterminals
* $\Sigma$: set of terminals (words)
* P: set of production rules
* <.>: start symbol

### A Mini-Generative-Grammar

1. <.> -> S
* S -> NP VP 
* NP -> N | DET N | DET ADJ N | PRP ADJ N
* VP -> V PP | V NP
* PP -> PREP PP | PREP NP
* PRP -> my | your | her
* DET -> a | the | this
* ADJ -> favorite | worst | dang
* N -> späti | cat | seminar | Toblerone
* V -> is | has | smells
* PREP -> out | of | like

##### Derivation

Format for each step: State after rule application (index of rule applied)

<.>

S (1)

NP VP (2)

PRP ADJ N VP (3)

PRP ADJ N V PP (4)

PRP ADJ N V PREP PP (5)

PRP ADJ N V PREP PREP NP (5)

PRP ADJ N V PREP PREP N (3)

My ADJ N V PREP PREP N (6)

My favorite N V PREP PREP N (8)

My favorite späti V PREP PREP N (9)

My favorite späti is PREP PREP N (10)

My favorite späti is out PREP N (11)

My favorite späti is out of N (11)

My favorite späti is out of Toblerone (9)



### Representing trees

* **linearized** - cf. [Vinyals et al 2015](https://arxiv.org/pdf/1412.7449.pdf) (Hinton's group at Google)

    (S (NP My favorite späti) (VP is (PP out (PP of (NP Toblerone)))) .)


* **action sequence** (this paper)

    Given 

    * *a* - a specific parsing algorithm (i.e. depth-first, left-to-right traversal),
    * *x* - a sequence of symbols, and
    * *y* - a parse tree,

    there is a unique sequence of actions

    \begin{align}
    a(x, y)
    \end{align}
    
    which can generate the tree.

#### Shift-reduce action sequence

* *SHIFT* : In parsing - *shift* word from buffer onto stack
    * $\rightarrow$ *GEN(x)* : in generating - *generate* new terminal *x*
* *NT(X)* : Open a nonterminal constituent of type X (i.e. NP, VP, ...)
* *REDUCE* : Close an open nonterminal constituent


In [ ]:
from get_oracle_gen import get_actions, get_tags_tokens_lowercase
example = "(S (NP (PRP My) (ADJ favorite) (N späti)) (VP (V is) (PP (PREP out) (PP (PREP of) (NP (N Toblerone))))) (. .))"

In [ ]:
actions = get_actions(example)
tags, tokens, lowercase = get_tags_tokens_lowercase(example)
i, j = 0, 0

In [ ]:
print("{}: {}".format(i, actions[i]))
i += 1

In [ ]:
print("{}: {}".format(j, tokens[j]))
j += 1

##### Parser architecture

Interim parse states are stored on the **stack**, which at any time *t* may contain:

* open nonterminals
* completed nonterminals
* terminals

The resulting sequences of actions and terminals are stored in the **action history** and **output buffer** respectively.

(In discriminative parsing, instead of an output buffer, we have an *input buffer* to read in the given symbol sequence.)

<img src="img/gen_derivation.png",width=700>

### RNNG (Recurrent Neural Network Grammar)

<!-- G = (N, Sig, P, S)-->

<!--img src="img/rnng_grammar.png",width=120,height=120-->

\begin{align}
G = (N, \Sigma, \Theta)
\end{align}

* N: set of nonterminals
* $\Sigma$: set of terminals (tokens: words, punctuation)
* $\Theta$: parameters learned by network

To ensure valid parse trees, a constrained set of valid parser actions are defined at time step *t*:

\begin{align}
A_G(S, n)
\end{align}

* *$S$*: state of the stack
* *n*: number of open nonterminals on the stack

#### Generative model

Learns a joint probability distribution of sentences and action sequences:

\begin{align}
p(x, y)& = \prod_{t=1}^{|a(x,y)|}p(a_t|\mathbf{a}_{\lt{t}}) \\
& = \prod_{t=1}^{|a(x,y)|}\frac{\exp \mathbf{r}_{at}^\top \mathbf{u}_t + b_{at}}{\sum_{a' \in A_G(S_t, n_t)} \exp \mathbf{r}_{a'}^\top \mathbf{u}_t + b_{a'}}
\end{align}

* **$r_a$**: embedding of the action $a \in (SHIFT, REDUCE, NT(X_1), ..., NT(X_n))$
* **$u_t$**: embedding of the algorithm context at time *t*

\begin{align}
\mathbf{u}_t = tanh(\mathbf{Ws}_t + \mathbf{c})
\end{align}

* **$s_t$**: embedding of the stack at time *t*

The original paper also incorporated embeddings of the output buffer - i.e. the sequence of words that had already been generated until time *t* - and the action history - the sequence of actions taken until time *t* - but [Kuncoro et al. 2017](http://www.aclweb.org/anthology/E17-1117) found that the RNNG performed best when trained on the stack embedding alone.

#### Discriminative model

Learns a conditional probability distribution of action sequences given a sentence.

The equation is the same as above, with three changes:

\begin{align}
 p(x, y) & \rightarrow p(y|x) \\
 A_G(S, n) & \rightarrow A_D(B, S, n) \\
 \mathbf{u}_t = tanh(\mathbf{Ws}_t + \mathbf{c}) & \rightarrow tanh(\mathbf{W}[\mathbf{s}_t;\mathbf{b}_t] + \mathbf{c})
\end{align}

* $p(y|x)$: conditional probability given the input sentence $x$
* $A_D$: set of valid parser actions for the discriminative model
    * $B$: input buffer containing the not-yet-parsed sequence of tokens in *x*
* $b_t$: embedding of the input buffer state at time *t*

##### Representing the stack: recursive NN composition function

As noted above, the stack can contain:

* open nonterminals
* completed nonterminals
* terminals

Terminals and open nonterminals can be treated as simple lookup parameters, so that's just a few more embeddings for the model to learn.

Completed nonterminals result from REDUCE operations, and they represent trees:

<img src="img/architecture_stack.png",width=300>

To obtain a representation of these trees, we need a **composition function** that can take sequences of arbitrary length.

Dyer et al. solve this problem by throwing a bidirectional LSTM at it:

<img src="img/stack_comp.png",width=500>

This gives us a **recursive neural network** approach to composition, as representations of terminals and subtrees are treated as equivalents in building the next tree.

##### Generating words: class-factored softmax

Hypothetically, the number of valid actions for the generative model to choose from would be:

\begin{align}
N + \Sigma + 1
\end{align}

which becomes very daunting when dealing with a large vocabulary.

But if word generation is one action, GEN/SHIFT, the set of valid actions is $N + 2$ :

\begin{align}
a \in (SHIFT, REDUCE, NT(X_1), ..., NT(X_n))
\end{align}

When the parser predicts GEN/SHIFT, word generation is handled in a second step using **class-factored softmax** where words are sampled based on a predetermined subclass.

## Demo

In [1]:
from rnng import Vocab, TransitionParser, main
import dynet as dy
import random

In [2]:
model, tp, train, dev = main()

### Walk through one tree-building example

In [3]:
which = random.randrange(len(train))
print(which)
t_inst = train[which]

724


In [4]:
def print_train_inst(t_inst):
  print(t_inst[0] + "\n\n" + " ".join(t_inst[1]) + "\n\n" + " ".join(t_inst[2]))
  
print_train_inst(t_inst)

# (FRAG (`` ``) (CC Or) (SBAR (IN if) (S  (NP (PRP they) ) (VP (VBP feel) (SBAR  (SBAR  (S  (NP (DT the) (NN wine) ) (VP (VBZ is) (VP (VBN overpriced) )))) (CC and) (SBAR  (S  (NP (PRP they) ) (VP (MD can) (VP (VB get) (NP  (NP (NN something) ) (ADJP (RB equally) (JJ good) )) (PP (IN for) (ADJP (JJR less) )))))))))) (. .) ('' '') )

`` Or if they feel the wine is overpriced and they can get something equally good for less . ''

NT(FRAG) SHIFT SHIFT NT(SBAR) SHIFT NT(S) NT(NP) SHIFT REDUCE NT(VP) SHIFT NT(SBAR) NT(SBAR) NT(S) NT(NP) SHIFT SHIFT REDUCE NT(VP) SHIFT NT(VP) SHIFT REDUCE REDUCE REDUCE REDUCE SHIFT NT(SBAR) NT(S) NT(NP) SHIFT REDUCE NT(VP) SHIFT NT(VP) SHIFT NT(NP) NT(NP) SHIFT REDUCE NT(ADJP) SHIFT SHIFT REDUCE REDUCE NT(PP) SHIFT NT(ADJP) SHIFT REDUCE REDUCE REDUCE REDUCE REDUCE REDUCE REDUCE REDUCE REDUCE REDUCE SHIFT SHIFT REDUCE


In [ ]:
stack, params = tp.gen_setup(.3)
terms, results, open_nts, losses = [], [], [], []

### start loop here

Executing each of the cells below in sequence will walk you through one parsing step for the network and expose intermediate states of the stack so you can inspect them.

In [ ]:
valid_actions = tp.get_valid_actions(stack, open_nts)
#[tp.vocab_acts.i2w[action] for action in valid_actions]

In [ ]:
action, loss = tp.get_action(stack, params, valid_actions, len(results), t_inst[2], .3)
losses.append(loss) if loss else loss
results.append(action)
tp.vocab_acts.i2w[action]

In [ ]:
term, nt_idx, loss = tp.do_action(stack, action, params, open_nts, len(terms), t_inst[1], .3)
losses.append(loss) if loss else loss
terms.append(term) if term else term
open_nts.append(nt_idx) if nt_idx else nt_idx 
term, nt_idx, loss

In [ ]:
stack

### stop loop here

In [ ]:
# train a model for one epoch - takes ~25 min on my machine
tp.train(train, dy.SimpleSGDTrainer(model), dev, epochs=1)

In [ ]:
# generate a sentence from the trained model
tp.generate()

In [ ]:
# save model
model.save("1epoch.model")

In [ ]:
# load model
model, tp, train, dev = main()
model.populate("1epoch.model")

In [ ]:
model.parameter_count()